In [1]:
from tensorflow.keras import *
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf

In [2]:
data_path = 'J:/VeReMi-Dataset/000_Select_data'
os.chdir(data_path)
datalist = os.listdir()

In [3]:
testX_regularized=np.load('testX_processed.npy')
testY=np.load('testY_processed.npy')
trainX_regularized=np.load('trainX_processed.npy')
trainY=np.load('trainY_processed.npy')
valX_regularized=np.load('valX_processed.npy')
valY=np.load('valY_processed.npy')

In [4]:
def get_info(bucket_list, data_label_df):
    result = []
    for i in data_label_df:
        result.append(i[bucket_list])
    return result

In [19]:
model = Sequential()
model.add(layers.Dense(30,activation='relu',input_shape=[30]))

model.add(layers.Dropout(0.8))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dropout(0.8))
model.add(layers.Dense(2, activation='sigmoid'))
model.compile(optimizer=optimizers.Adam(),
              loss=losses.binary_crossentropy,
              metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 30)                930       
_________________________________________________________________
dropout_4 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                496       
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 34        
Total params: 1,460
Trainable params: 1,460
Non-trainable params: 0
_________________________________________________________________


In [7]:
trainY=tf.one_hot(trainY,2)
valY=tf.one_hot(valY,2)
testY_m=tf.one_hot(testY,2)

In [21]:
history = model.fit(trainX_regularized, trainY, batch_size=64, epochs=3, validation_data=(valX_regularized, valY))

Epoch 1/3
27407/27407 [==============================] - 21s 760us/step - loss: 0.3304 - accuracy: 0.9970 - val_loss: 0.2879 - val_accuracy: 1.0000

In [22]:
effect = model.evaluate(testX_regularized, testY_m)

7000/7000 [==============================] - 3s 442us/step - loss: 0.3292 - accuracy: 1.0000

In [23]:
#testX_regularized
y_pred=model.predict(testX_regularized,batch_size=1)
y_pred_label=[]
for i in y_pred:
    if i[0]>i[1]:
        y_pred_label.append(0.0)
    else:
        y_pred_label.append(13.0)

In [24]:
right=0
for loc in range (223972):
    if testY[loc]==y_pred_label[loc]*1.0:
        right+=1
acc=right/223972
print(acc)

0.6453440608647509
